In [ ]:
rm(list = ls())

library(data.table)
library(tidyverse)
library(rJava)
library(RNetLogo)

library(lhs)  # For maximin Latin hypercube sampling
library(ggplot2)
library(plotly)  # For beautiful plotting
library(caret)
library(randomForest)
library(factoextra)
library(e1071)
library(TSrepr)  # for evaluating predictive power

require(gridExtra)

options(warn = -1)

In [ ]:
nl.model <- "Segregation"
folder.path = "C:/Users/paslanpatir/Desktop/TEZ_v2/"

model.type = ifelse(nl.model == "Segregation", "basic", "dummy")
# the path of data folder
data.path = paste0(folder.path,"data/")


In [ ]:
train_ins_oneshot = 100
metarep = c(1,2,3,4,5)
seed.oneshot = c(0,1,2,3,4,5,6,7,8,20)

In [ ]:
#### Data Set ####
training_set_all = data.table()

for( i in seed.oneshot){
    
    training_set.name= paste0(data.path,"training_set","_",model.type,"_",train_ins_oneshot,"_seed",i,".csv")
    training_set <- fread(training_set.name) 
    
    assign(paste0("training_set_",i),training_set)
    
    training_set_all = rbind(training_set_all,data.table(training_set, "seed" = i))
    rm(training_set,training_set.name)    
}

one_shot_data = copy(training_set_all)

# e1071

## Leave-one-out

In [ ]:
## tune.RandomForest ( e1071 )
 print(paste0(" Tuning start time : ", Sys.time()))
       
for (i in seed.oneshot) {
    
    training_set = copy(one_shot_data[seed == i, .SD, .SDcols = -c("seed")])
    for (r in metarep) {

    training_set.name= paste0(data.path,"training_set","_",model.type,"_",train_ins_oneshot,"_seed",i,".csv")
    training_set <- fread(training_set.name) 

    tuning = tune.randomForest(x = training_set[, -c("output")]
                             , y = training_set$output
                             , ntree = c(10,50,100,200,300,400)
                             , mtry = c(1,2)
                             , tunecontrol = tune.control(sampling = "cross", cross = length(training_set$output)))

tune = data.table("seed" = i ,"rep" = r
, ntree = tuning$best.parameters$ntree 
, mtry = tuning$best.parameters$mtry 
, best_perf = tuning$best.performance ) 

            fwrite(tune,paste0(folder.path,model.type,"_train",train_ins_oneshot,"_tune",".csv") ,append = TRUE)

print(paste0("seed = ",i,"    rep = ",r))

}
}
print(paste0(" Tuning end time : ", Sys.time()))

# 10-fold CrossValidation


In [ ]:
## tune.RandomForest ( e1071 )
 print(paste0(" Tuning start time : ", Sys.time()))
       
for (i in seed.oneshot) {
    
    training_set = copy(one_shot_data[seed == i, .SD, .SDcols = -c("seed")])
    for (r in metarep) {

    training_set.name= paste0(data.path,"training_set","_",model.type,"_",train_ins_oneshot,"_seed",i,".csv")
    training_set <- fread(training_set.name) 

    tuning = tune.randomForest(x = training_set[, -c("output")]
                             , y = training_set$output
                             , ntree = c(10,50,100,200,300,400)
                             , mtry = c(1,2)
                             , tunecontrol = tune.control(sampling = "cross", cross = 10))

tune = data.table("seed" = i ,"rep" = r
, ntree = tuning$best.parameters$ntree 
, mtry = tuning$best.parameters$mtry 
, best_perf = tuning$best.performance ) 

            fwrite(tune,paste0(folder.path,model.type,"_10Fold_train",train_ins_oneshot,"_tune",".csv") ,append = TRUE)

print(paste0("seed = ",i,"    rep = ",r))

}
}
print(paste0(" Tuning end time : ", Sys.time()))

# randomForest 

In [ ]:
# print(paste0(" Tuning start time : ", Sys.time()))
#       
#for (i in seed.oneshot) {
#    
#    training_set = copy(one_shot_data[seed == i, .SD, .SDcols = -c("seed")])
#    for (r in metarep) {
#
#    training_set.name= paste0(data.path,"training_set","_",model.type,"_",train_ins_oneshot,"_seed",i,".csv")
#    training_set <- fread(training_set.name) 
#
#    tuning = tune.randomForest(x = training_set[, -c("output")]
#                             , y = training_set$output
#                             , mtryStart = 2
#                             , ntreeTry=50
#                             , stepFactor=2
#                             , improve=0.05
#                             ,trace=TRUE, plot=FALSE, doBest=TRUE)
#
#tune = data.table("seed" = i ,"rep" = r
#, ntree = tuning$best.parameters$ntree 
#, mtry = tuning$best.parameters$mtry 
#, best_perf = tuning$best.performance ) 
#
#            fwrite(tune,paste0(folder.path,model.type,"_train100_tune",".csv") ,append = TRUE)
#
#print(paste0("seed = ",i,"    rep = ",r))
#
#}
#}
#print(paste0(" Tuning end time : ", Sys.time()))